# 誤差逆伝播法の実装

## ニューラルネットワークの学習の全体図

以下、斎藤（2016）より引用。

### 前提
* ニューラルネットワークは、適応可能な重みとバイアスがあり、この重みとバイアスを訓練データに適応するように調整することを「学習」と呼ぶ。
* ニューラルネットワークの学習は、次の4つの手順で行う。

### ニューラルネットワークの学習手順

1. ミニバッチ
* 訓練データの中からランダムに一部のデータを選び出す。

2. 勾配の計算
* 各重みパラメータに関する損失関数の勾配を求める。

3. パラメータの更新
* 重みパラメータを勾配方向に微小量だけ更新する。

4. 繰り返す
以上のステップ1～3を繰り返す。

In [2]:
# 実験
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layer import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [5]:

def TwoLayerNet(input_size, hidden_size, output_size, weight_init_std = 0.01):
    params = {}
    params["W1"] = weight_init_std * \
                    np.random.randn(input_size, hidden_size)
    params["b1"] = np.zeros(hidden_size)
    params["W2"] = weight_init_std * \
                    np.random.randn(hidden_size, output_size)
    params["b2"] = np.zeros(output_size)

    # レイヤの生成
    layers = OrderedDict()         # 順序付きディクショナリ型の空データを用意
    layers["Affine1"] = \
    Affine(params["W1"], params["b1"])
    layers["Relu1"] = Relu()
    layers["Affine2"] = \
    Affine(params["W2"], params["b2"])

    lastLayer = SoftmaxWithLoss()

    return params, layers, lastLayer

In [6]:
params, layers, lastLayer = TwoLayerNet(784, 50, 10)

In [13]:
layers.values()

odict_values([<common.layer.Affine object at 0x000001A16FF7C110>, <common.layer.Relu object at 0x000001A17E68BDA0>, <common.layer.Affine object at 0x000001A17E68B560>])

In [10]:
layers

OrderedDict([('Affine1', <common.layer.Affine at 0x1a16ff7c110>),
             ('Relu1', <common.layer.Relu at 0x1a17e68bda0>),
             ('Affine2', <common.layer.Affine at 0x1a17e68b560>)])